In [1]:
import os
import numpy as np
import pandas as pd
from IPython.display import display
from IPython.core.display import HTML
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import Div
from bokeh.layouts import row

output_notebook()
pd.options.display.float_format = '{:,.5f}'.format
HTML('<link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css">')

Loading BokehJS ...

Backtesting of the proposal 11487
---

In [2]:
from terra_algo_backtest.binance_loader import new_binance_client

# Replace these with your Binance API key and secret
client = new_binance_client(os.getenv("BINANCE_API_KEY"), os.getenv("BINANCE_API_SECRET"))

In [7]:
from binance import Client
from terra_algo_backtest.utils import format_df
from terra_algo_backtest.plotting import new_trade_figure, plot_price_ratio

mkt_pair = 'USTC/BUSD'
start, end = '2023-01-01 00:00:00', '2023-06-30 23:59:59'
frequency = Client.KLINE_INTERVAL_1HOUR

qty_factor = 0.01
df_trades = client.get_trade_data(mkt_pair, frequency, start, end)
df_trades["quantity"] = qty_factor*df_trades["quantity"]
show(Div(text=format_df(df_trades.head(10), width=800)))

In [9]:
from terra_algo_backtest.market import MarketQuote, new_market
from terra_algo_backtest.simulation import swap_simulation
from terra_algo_backtest.plotting import new_simulation_figure
from terra_algo_backtest.plot_layout import div_layout, new_div_layout
from terra_algo_backtest.exec_engine import ConstantProductEngine
from terra_algo_backtest.strategy import SimpleUniV2Strategy, DivProtocolStrategy, DivProtocolParams

liquidity_usd = 100000
# USTC/BUSD market price
base = MarketQuote("USTC/BUSD", df_trades.price.iloc[0])
# BUSD/USD market price
quote = MarketQuote("BUSD/USD", 1)
# create a 1,000,000 USD market for USTC/BUSD with 0.3% swap fee 
mkt = new_market(liquidity_usd, quote, base, 0.001)
# create a cp swap execution engine
cp_amm = ConstantProductEngine(mkt)
# div protocol with buy back strategy on top the amm strat
strategy = DivProtocolStrategy(
    strategy=SimpleUniV2Strategy(cp_amm, arb_enabled=True),
    strat_params=DivProtocolParams(peg_price=0.015),
)
# run simulation
simul = swap_simulation(df_trades, strategy)
# display results
show(new_simulation_figure(mkt, simul, new_div_layout(simul["breakdown"])))

Function 'trade_summary' executed in 0.0077s
Function 'sim_results' executed in 0.1227s
Function 'swap_simulation' executed in 0.5191s
